# Exploring and clustering neighborhoods in Toronto - Part 2
Coursera capstone course for applied data science

Here we create a dataframe adding the coordinates for each postal code within Toronto

In [45]:
# we install the geocoder package
!pip install geocoder

In [33]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from IPython.display import display_html

Here we load the table using requests and BeautifulSoup.  
The first table with the class "wikitable" is the one we are looking for.  
An interesting Beautiful Soup tutorial can be found here https://www.youtube.com/watch?v=ng2o98k983k

In [34]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url).content
soup = BeautifulSoup(html, 'lxml')
table = soup.find('table', class_='wikitable')

Now, we load the dataframe using the read_html method from the IPython library.    
Next lines inspired from this tutorial 
https://beenje.github.io/blog/posts/parsing-html-tables-in-python-with-pandas/

In [35]:
# render the table from the HTML string
# display_html(str(table), raw=True)
# load the table in a dataframe
df = pd.read_html(str(table),header=0)[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Table cleanup

In [36]:
# rename the columns
df.columns = ['PostalCode', 'Borough', 'Neighborhood']

# delete the rows where the Borough is not assigned
df = df[df.Borough != 'Not assigned']

# group by Borough and combine the Neighborhood
# see here https://stackoverflow.com/questions/17841149/pandas-groupby-how-to-get-a-union-of-strings
df = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: "%s" % ', '.join(x)).reset_index()

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
# https://stackoverflow.com/questions/35068722/pandas-assign-value-of-one-column-based-on-another
df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned', df['Borough'], df['Neighborhood'])
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Finally we print the dimensions of the table

In [37]:
df.shape

(103, 3)

Adding the coordinates to the dataframe using the CSV file using the IBM Watson code genearted automatically to load files from the project assets.  
See heere how to load the file from the project assets: 

https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/load-and-access-data.html

NOTE: The cell to achieve that contains some sensitive data, so it has been marked as such and that code won´t be visible.


In [38]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


After loading the coordinates in the dataframe, we need to join the two dataframes using the column PostalCode as key

In [39]:
# load csv file with the coords
df_coords = df_data_1
df_coords.rename(columns={'Postal Code':'PostalCode'}, inplace=True)

# merge the two dataframes
# https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
df = pd.merge(df, df_coords, on='PostalCode')

Let´s check the result

In [44]:
df.head(50)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
